In [49]:
import pandas as pd
import numpy as np
from scipy.fftpack import fft
from scipy.stats import norm
import pywt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from tqdm import tqdm
from sklearn.utils import all_estimators
import aeon

import warnings
import xgboost

# import catboost
import lightgbm

warnings.filterwarnings("ignore")
pd.set_option("display.precision", 2)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

def sax_transform(series, w, a):
    paa = [series[i:i + w].mean() for i in range(0, len(series), w)]
    
    # Check if the standard deviation is zero
    if np.std(paa) != 0:
        paa = (paa - np.mean(paa)) / np.std(paa)
    else:
        # If standard deviation is zero, just subtract the mean
        paa = paa - np.mean(paa)
    
    breakpoints = norm.ppf(np.linspace(0, 1, a+1)[1:-1])
    sax_symbols = np.array(range(a))
    sax_representation = sax_symbols[np.digitize(paa, breakpoints)]
    
    return sax_representation

# The function has been adjusted to avoid the division by zero warning.

def transform_data(X):
    a = 4
    w = 9
    

    X_sax = np.array([sax_transform(row, w, a) for row in X])
    X_fft = np.abs(fft(X, axis=1))
    
    coeffs_cA, coeffs_cD = pywt.dwt(X, 'db1', axis=1)
    X_dwt = np.hstack((coeffs_cA, coeffs_cD))
    
    X_paa = np.column_stack([X[:, i:i+2].mean(axis=1) for i in range(0, X.shape[1], 2)])

    return {
        "TS": X,
        "FFT": X_fft,
        "DWT": X_dwt,
        "PAA": X_paa,
        "SAX": X_sax
    }

def select_model(option, random_state):
    all_est = all_estimators(type_filter='classifier')  # Filtra apenas os classificadores

    for name, ClassifierClass in all_est:
        if option.lower() in name.lower():
            if option.lower() == 'svm':
                return ClassifierClass(probability=True)
            elif option.lower() == 'gbc':
                return ClassifierClass(n_estimators=100, random_state=random_state)
            elif option.lower() == 'xgb':
                return ClassifierClass(random_state=random_state)
            else:
                return ClassifierClass()

    # Se nenhum correspondente for encontrado, retorne o RandomForestClassifier como padrão
    return RandomForestClassifier(n_estimators=100, random_state=random_state)

def train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='1nn'):
    trained_models = {}  # To store trained models for each transformation
    
    X_train_transformed = transform_data(X_train)  # Transform the entire training set once

    # Train a model for each transformation and store it
    for rep, X_trans in tqdm(X_train_transformed.items(), desc="Training Base Models"):
        model = select_model(base_option)
        model.fit(X_trans, y_train)
        trained_models[rep] = model
        
    # Prepare data for the meta-classifier
    meta_features = []
    for i in range(X_train.shape[0]):
        instance_features = []
        for rep, model in trained_models.items():
            proba = model.predict_proba(X_train_transformed[rep][i].reshape(1, -1))
            instance_features.extend(proba[0])
        meta_features.append(instance_features)
    
    meta_features = np.array(meta_features)
    
    # Train the meta-classifier
    meta_classifier = select_model(meta_option)
    meta_classifier.fit(meta_features, y_train)
    
    return trained_models, meta_classifier

def predict_with_meta_classifier(X_test, trained_base_models, trained_meta_classifier):
    predictions = []
    
    for i in tqdm(range(len(X_test)), desc="Testing Instances"):
        x_instance = X_test[i].reshape(1, -1)
        x_transformed = transform_data(x_instance)
        
        instance_features = []
        for rep, model in trained_base_models.items():
            proba = model.predict_proba(x_transformed[rep])
            instance_features.extend(proba[0])
        
        meta_feature = np.array(instance_features).reshape(1, -1)
        
        # Use the meta-classifier to predict
        predicted_value = trained_meta_classifier.predict(meta_feature)[0]
        predictions.append(predicted_value)
    
    return predictions



In [71]:
try:
    train_data = pd.read_csv('/DS/CSV_Qualificação/Beef_TRAIN.csv')
    test_data = pd.read_csv('/DS/CSV_Qualificação/Beef_TEST.csv')
except FileNotFoundError:
    print("Ensure the CSV files are in the correct path.")
    raise

X_train = train_data.drop('target', axis=1).values
y_train = train_data['target'].values

X_test = test_data.drop('target', axis=1).values
y_test = test_data['target'].values

In [72]:
from sklearn.utils import all_estimators
from sklearn.base import ClassifierMixin
from sklearn.metrics import accuracy_score

removed_classifiers = [
    "ClassifierChain",
    "ComplementNB",
    "GradientBoostingClassifier",
    "GaussianProcessClassifier",
    "HistGradientBoostingClassifier",
    "MLPClassifier",
    "LogisticRegressionCV",
    "MultiOutputClassifier",
    "MultinomialNB",
    "OneVsOneClassifier",
    "OneVsRestClassifier",
    "OutputCodeClassifier",
    "RadiusNeighborsClassifier",
    "VotingClassifier"
    "CategoricalNB",
    "LinearSVC"
]

CLASSIFIERS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], ClassifierMixin) and (est[0] not in removed_classifiers))
]

# Lists to store results
model_names = []
test_accuracies = []

for name, ClassifierClass in CLASSIFIERS:
    try:
        algos = ClassifierClass()

        #print(f'Meta-Classifier with Base Model {name.upper()}')

        # Training (update the function arguments based on your implementation)
        trained_base_models, meta_classifier = train_with_meta_classifier(X_train, y_train, base_option=name, meta_option=name)

        # Testing (update the function arguments based on your implementation)
        predictions_test_meta = predict_with_meta_classifier(X_test, trained_base_models, meta_classifier)

        test_accuracy_meta = accuracy_score(y_test, predictions_test_meta)
        
        # Collect results
        model_names.append(name)
        test_accuracies.append(test_accuracy_meta)

        #print(f'Test Accuracy for Meta-Classifier with Base Model {name}: {test_accuracy_meta}')
    except Exception as e:
        print(f"An error occurred while training or testing with {name}: {e}")
    print("-------------------------------")

# Create a DataFrame
scores = pd.DataFrame({
    "Model": model_names,
    "Accuracy": test_accuracies
})

# Sort and set the index
scores = scores.sort_values(by="Accuracy", ascending=False).set_index("Model")

# Display the DataFrame
print(scores)


Testing Instances: 100%|██████████| 30/30 [00:01<00:00, 23.70it/s]


-------------------------------


Testing Instances: 100%|██████████| 30/30 [00:00<00:00, 84.98it/s]


-------------------------------


Testing Instances: 100%|██████████| 30/30 [00:00<00:00, 189.18it/s]


...

An error occurred while training or testing with RidgeClassifierCV: 'RidgeClassifierCV' object has no attribute 'predict_proba'
-------------------------------


Training Base Models: 100%|██████████| 5/5 [00:00<00:00, 15.27it/s]


An error occurred while training or testing with SGDClassifier: probability estimates are not available for loss='hinge'
-------------------------------


Training Base Models: 100%|██████████| 5/5 [00:00<00:00,  9.90it/s]

An error occurred while training or testing with SVC: 'LinearSVC' object has no attribute 'predict_proba'
-------------------------------
An error occurred while training or testing with StackingClassifier: StackingClassifier.__init__() missing 1 required positional argument: 'estimators'
-------------------------------
An error occurred while training or testing with VotingClassifier: VotingClassifier.__init__() missing 1 required positional argument: 'estimators'
-------------------------------
                               Accuracy
Model                                  
CalibratedClassifierCV             0.87
GaussianNB                         0.83
LogisticRegression                 0.83
BaggingClassifier                  0.77
ExtraTreesClassifier               0.73
RandomForestClassifier             0.70
DecisionTreeClassifier             0.63
AdaBoostClassifier                 0.60
KNeighborsClassifier               0.60
ExtraTreeClassifier                0.50
LinearDiscriminant

In [57]:
from aeon.registry import all_estimators
all=all_estimators(as_dataframe=True, estimator_types='classifier')
all

,name,estimator
0,Arsenal,<class 'aeon.classification.convolution_based....
1,BOSSEnsemble,<class 'aeon.classification.dictionary_based._...
2,CNNClassifier,<class 'aeon.classification.deep_learning._cnn...
3,CanonicalIntervalForestClassifier,<class 'aeon.classification.interval_based._ci...
4,Catch22Classifier,<class 'aeon.classification.feature_based._cat...
5,ChannelEnsembleClassifier,<class 'aeon.classification.compose._channel_e...
6,ClassifierPipeline,<class 'aeon.classification.compose._pipeline....
7,ContractableBOSS,<class 'aeon.classification.dictionary_based._...
8,DrCIFClassifier,<class 'aeon.classification.interval_based._dr...
9,DummyClassifier,<class 'aeon.classification._dummy.DummyClassi...
